## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-14-23-41-19 +0000,nyt,Suspect Pleads Guilty in Firebombing of Pennsy...,https://www.nytimes.com/2025/10/14/us/arson-ha...
1,2025-10-14-23-39-23 +0000,nyt,Share Your Videos of ICE Interactions,https://www.nytimes.com/2025/10/14/us/videos-i...
2,2025-10-14-23-35-24 +0000,cbc,Stellantis says it will invest $13B to expand ...,https://www.cbc.ca/news/canada/windsor/stellan...
3,2025-10-14-23-26-28 +0000,latimes,Former Rep. Katie Porter expresses remorse abo...,https://www.latimes.com/california/story/2025-...
4,2025-10-14-23-21-38 +0000,nyt,Trump Dangles $20 Billion Lifeline for Argenti...,https://www.nytimes.com/2025/10/14/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,42
262,gaza,15
261,hamas,15
406,china,14
13,will,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
193,2025-10-14-15-25-15 +0000,nypost,Hamas won’t give up arms or Gaza control — des...,https://nypost.com/2025/10/14/world-news/hamas...,103
156,2025-10-14-17-30-22 +0000,nypost,"Gaza peace process ‘IS NOT DONE,’ Trump warns ...",https://nypost.com/2025/10/14/us-news/gaza-pea...,99
209,2025-10-14-14-24-22 +0000,nypost,Kamala Harris refuses to name Trump as she lau...,https://nypost.com/2025/10/14/us-news/kamala-h...,98
122,2025-10-14-18-52-56 +0000,nypost,"Hamas will disarm or ‘we will disarm them,’ Tr...",https://nypost.com/2025/10/14/us-news/hamas-wi...,98
208,2025-10-14-14-33-51 +0000,nyt,How Trump’s Gaza Peace Plan Presents Hurdles f...,https://www.nytimes.com/2025/10/14/world/middl...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
193,103,2025-10-14-15-25-15 +0000,nypost,Hamas won’t give up arms or Gaza control — des...,https://nypost.com/2025/10/14/world-news/hamas...
333,60,2025-10-14-01-00-00 +0000,wsj,"The White House move to lay off roughly 4,000 ...",https://www.wsj.com/politics/policy/trump-gove...
230,53,2025-10-14-13-00-00 +0000,wsj,"Tariffs, inflation and other threats such as e...",https://www.wsj.com/economy/trade-tensions-wei...
327,52,2025-10-14-01-59-18 +0000,cbc,"New York Times, AP, Reuters and Newsmax among ...",https://www.cbc.ca/news/world/new-york-times-a...
108,49,2025-10-14-19-42-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
115,41,2025-10-14-19-08-39 +0000,bbc,Israel identifies bodies of first four dead ho...,https://www.bbc.com/news/articles/cx27ye11ey2o...
122,37,2025-10-14-18-52-56 +0000,nypost,"Hamas will disarm or ‘we will disarm them,’ Tr...",https://nypost.com/2025/10/14/us-news/hamas-wi...
211,35,2025-10-14-14-20-00 +0000,wsj,China’s Rare-Earth Export Controls Are ‘Critic...,https://www.wsj.com/economy/trade/chinas-rare-...
101,34,2025-10-14-20-01-24 +0000,nyt,U.S. Military Kills 6 People in Boat Attack Ne...,https://www.nytimes.com/2025/10/14/us/politics...
190,32,2025-10-14-16-00-00 +0000,wsj,He Works Full Time and Goes to College. Is He ...,https://www.wsj.com/personal-finance/savings/c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
